In [1]:
import os
from tqdm import tqdm
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf

import sys

sys.path.append("../")
from obj_detection import model

In [2]:
print(
    "GPU is available" if tf.test.gpu_device_name() else "GPU is NOT available"
)
if tf.test.gpu_device_name():
    physical_devices = tf.config.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

GPU is NOT available


In [3]:
annotations_path = "../data/annotations/"
images_path = "../data/images/"

In [4]:
annotations = os.listdir(annotations_path)
images = os.listdir(images_path)

In [5]:
img_name_list = []
width_list = []
height_list = []
label_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []

In [6]:
for i in tqdm(range(len(annotations))):
    tree = ET.parse(os.path.join(annotations_path, annotations[i]))
    root = tree.getroot()
    img_name = root.find("filename").text

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    for group in root.findall("object"):
        label = group.find("name").text
        bbox = group.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        img_name_list.append(img_name)
        width_list.append(width)
        height_list.append(height)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        xmax_list.append(xmax)
        ymax_list.append(ymax)
        label_list.append(label)

100%|██████████| 877/877 [00:08<00:00, 100.38it/s]


In [7]:
dataset = pd.DataFrame(
    {
        "img_name": img_name_list,
        "width": width_list,
        "height": height_list,
        "xmin": xmin_list,
        "ymin": ymin_list,
        "xmax": xmax_list,
        "ymax": ymax_list,
        "label": label_list,
    }
)
dataset.head()

,img_name,width,height,xmin,ymin,xmax,ymax,label
0,road0.png,267,400,98,62,208,232,trafficlight
1,road1.png,400,283,154,63,258,281,trafficlight
2,road10.png,400,267,106,3,244,263,trafficlight
3,road100.png,400,385,35,5,363,326,speedlimit
4,road101.png,400,200,195,7,392,194,speedlimit


In [8]:
classes = dataset["label"].unique().tolist()
classes

['trafficlight', 'speedlimit', 'crosswalk', 'stop']

In [9]:
label_encoder = LabelBinarizer()

In [10]:
images = []
labels = []
bboxes = []
for index, row in dataset.iterrows():
    img_path = os.path.join(images_path, row["img_name"])
    img = cv2.imread(img_path)
    x_scale = 224.0 / int(row["width"])
    y_scale = 224.0 / int(row["height"])
    img = cv2.resize(img, (224, 224))
    images.append(img)

    xmin = int(row["xmin"] * x_scale)
    ymin = int(row["ymin"] * y_scale)
    xmax = int(row["xmax"] * x_scale)
    ymax = int(row["ymax"] * y_scale)
    bboxes.append((xmin, ymin, xmax, ymax))
    labels.append(row["label"])

In [11]:
images = np.array(images, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
labels = label_encoder.fit_transform(labels)

In [12]:
(
    train_images,
    val_images,
    train_bboxes,
    val_bboxes,
    train_labels,
    val_labels,
) = train_test_split(images, bboxes, labels, test_size=0.2, random_state=12)

In [13]:
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}

train_targets = {
    "class_label": train_labels,
    "bounding_box": train_bboxes,
}

val_targets = {
    "class_label": val_labels,
    "bounding_box": val_bboxes,
}

In [14]:
base_learning_rate = 0.0001

# opt = RMSprop(learning_rate=base_learning_rate)
opt = Adam(base_learning_rate)

In [15]:
ssd = model.SSD_Model(
    num_classes=len(classes),
    input_size=(224, 224, 3),
    optimizer=opt,
    losses=losses,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_images (InputLayer)      [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_images[0][0]']           
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [16]:
ssd.model_fit(
    train_images=train_images,
    train_targets=train_targets,
    val_images=val_images,
    val_targets=val_targets,
    epochs=15,
    batch_size=25,
)

Epoch 1/15
40/40 [==============================] - 71s 2s/step - loss: 1.1191 - bounding_box_loss: 0.0354 - class_label_loss: 1.0837 - bounding_box_accuracy: 0.4543 - class_label_accuracy: 0.6101 - val_loss: 0.8289 - val_bounding_box_loss: 0.0292 - val_class_label_loss: 0.7997 - val_bounding_box_accuracy: 0.4779 - val_class_label_accuracy: 0.6827
Epoch 2/15
40/40 [==============================] - 60s 2s/step - loss: 0.8433 - bounding_box_loss: 0.0311 - class_label_loss: 0.8122 - bounding_box_accuracy: 0.5156 - class_label_accuracy: 0.6824 - val_loss: 0.5874 - val_bounding_box_loss: 0.0247 - val_class_label_loss: 0.5627 - val_bounding_box_accuracy: 0.5984 - val_class_label_accuracy: 0.8072
Epoch 3/15
40/40 [==============================] - 60s 2s/step - loss: 0.6687 - bounding_box_loss: 0.0276 - class_label_loss: 0.6411 - bounding_box_accuracy: 0.5638 - class_label_accuracy: 0.7749 - val_loss: 0.5554 - val_bounding_box_loss: 0.0232 - val_class_label_loss: 0.5322 - val_bounding_box_ac

In [17]:
ssd.model.save("../models/output/model.h5")
ssd.model.save_weights("../models/output/model_weight.h5")